# Compute efficiency distributions for a proton sample

In [ ]:
#!python3 -m pip install --user uproot4 awkward1 mplhep

In [1]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import mplhep

### Proton reconstruction flavours

We will look at two ways to reconstruct protons in PPS: _single-RP_ and _multi-RP_

The flag below controls which proton reconstruction is used.

In the multi-RP reconstruction, both "near" and "far" tracking detector stations in one spectrometer arm are used. 

In the single-RP reconstruction, the proton kinematics is only partially reconstructed from a single detector station. In this case we will use the "far" station. In the 2017 detector configuration, this corresponds to the stations using pixel detectors.

In [2]:
fileName_data = "data/output-UL2017B-PreSel.h5"
#fileName_data = "data/output-UL2017C1-PreSel.h5"
#fileName_data = "data/output-UL2017F1-PreSel.h5"
#fileNames_data = [
#    'data/output-UL2017B-PreSel.h5',
#    'data/output-UL2017C1-PreSel.h5',
#    'data/output-UL2017F1-PreSel.h5'
#]

#proton_selection = "SingleRP"
proton_selection = "MultiRP"

## Access the data

In [4]:
df_signal = None
event_counts_signal = None
selections_str_signal = None

with h5py.File( fileName_data, 'r' ) as f:
    print ( list(f.keys()) )
    dset = f['protons']
    print ( dset.shape )
    print ( dset[:,:] )
    dset_columns = f['columns']
    print ( dset_columns.shape )
    columns = list( dset_columns )
    print ( columns )
    columns_str = [ item.decode("utf-8") for item in columns ]
    print ( columns_str )
    
    dset_counts = f['event_counts']
    event_counts_signal = list( dset_counts )
    print ( event_counts_signal )
    
    dset_selections = f['selections']
    selections_ = list( dset_selections )
    print ( selections_ )
    selections_str_signal = [ item.decode("utf-8") for item in selections_ ]
    print ( selections_str_signal )
    
    df_signal = pd.DataFrame( dset, columns=columns_str )
    
    df_signal = df_signal[ ['Run', 'LumiSection', 'EventNum', 'CrossingAngle', 
                            'MultiRP', 'Arm', 'RPId1', 'RPId2', 'TrackX1', 'TrackY1', 'TrackX2', 'TrackY2',
                            'Xi', 'T', 'ThX', 'ThY',
                            'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMuPlus', 'XiMuMuMinus'] ].astype( { "Run": "int64", "LumiSection": "int64", "EventNum": "int64", "MultiRP": "int32", "Arm": "int32", "RPId1": "int32", "RPId2": "int32", "ExtraPfCands": "int32" } )
    

['columns', 'event_counts', 'protons', 'selections']
(107075, 38)
[[2.97227000e+05 5.30000000e+01 2.29700000e+03 ... 9.83012105e-01
  5.61954090e-02 4.09129155e-03]
 [2.97227000e+05 5.50000000e+01 8.67000000e+02 ... 1.32611728e-01
  1.95323438e-03 4.83462101e-02]
 [2.97227000e+05 5.50000000e+01 8.67000000e+02 ... 1.32611728e-01
  1.95323438e-03 4.83462101e-02]
 ...
 [2.99062000e+05 1.33000000e+02 1.72100000e+03 ... 6.64818845e-01
  3.52177158e-03 7.56784159e-02]
 [2.99062000e+05 1.33000000e+02 1.72100000e+03 ... 6.64818845e-01
  3.52177158e-03 7.56784159e-02]
 [2.99062000e+05 1.33000000e+02 1.72100000e+03 ... 6.64818845e-01
  3.52177158e-03 7.56784159e-02]]
(38,)
[b'Run', b'LumiSection', b'BX', b'EventNum', b'CrossingAngle', b'MultiRP', b'Arm', b'RPId1', b'RPId2', b'TrackX1', b'TrackY1', b'TrackX2', b'TrackY2', b'Xi', b'T', b'ThX', b'ThY', b'Time', b'TrackThX_SingleRP', b'TrackThY_SingleRP', b'Track1ThX_MultiRP', b'Track1ThY_MultiRP', b'Track2ThX_MultiRP', b'Track2ThY_MultiRP', b'Muon0

### Proton efficiency evaluation

We will evaluate the efficiency correction for _multi-RP_ protons. 

This has to be computed as the product of the efficiency of the "near" detector station (silicon strips, in 2017) and the "multiRP efficiency".

The efficiency components for the near RP station is divided in two parts: multitrack and sensor. 
The former accounts the inefficiency caused by strips detectors not being abla to reconstruct multiple tracks in the same event. This is a single numeric factor.
The sensor efficiency takes into account local inefficiencies caused, e.g., by radiation damage and is supplied as a 2D, xy map. 

The multiRP efficiency accounts instead for both the sensor efficiency of the "far" RP (pixels in 2017 and 2018) and the inefficiency component given by the proton chance of interacting while travelling from the near to the far RP. Again, this is supplied as a 2D, xy map.

A different measurement is available for each data period here included: 2017B, 2017C1, 2017F1.

Despite being counter-intuitive, it should be noted that, for near RP efficiencies one should use the xy coordinates measured in the far RP (TrackX2/TrackY2) and for multiRP the ones measured in the near RP (TrackX1/TrackY1). This is a consequence of the Tag&Probe method used to produce the efficiency corrections.

In [8]:
import ROOT

# Efficiency correction files
strips_efficiency_file = ROOT.TFile.Open("/eos/project/c/ctpps/subsystems/Strips/StripsTracking/PreliminaryEfficiencies_July132020_1D2DMultiTrack.root")
multiRP_efficiency_file = ROOT.TFile.Open("/eos/project/c/ctpps/subsystems/Pixel/RPixTracking/pixelEfficiencies_multiRP.root")

data_taking_period = "2017B"
year = data_taking_period[:4]

strips_multitrack_efficiency = {}
strips_sensor_efficiency = {}
multiRP_efficiency = {}

# Retrieve histograms from files and save them in dictionaries for future usage
for sector in ["45","56"]:
    rp_number = "3" if sector == "45" else "103"
    strips_multitrack_efficiency[sector] = strips_efficiency_file.Get("Strips/"+year+"/"+data_taking_period+"/h"+sector+"multitrackeff_"+data_taking_period+"_avg_RP"+rp_number).GetBinContent(1)
    strips_sensor_efficiency[sector] = strips_efficiency_file.Get("Strips/"+year+"/"+data_taking_period+"/h"+sector+"_"+data_taking_period+"_all_2D")
    multiRP_efficiency[sector] = multiRP_efficiency_file.Get("Pixel/"+year+"/"+data_taking_period+"/h"+sector+"_220_"+data_taking_period+"_all_2D")

In [9]:
# Select only multiRP protons from dataframe
df_multiRP = df_signal[df_signal["MultiRP"] == 1]

# Create column for the efficiency
efficiency = []
for row_n,row in df_multiRP.iterrows():
    sector = "45" if row["Arm"] == 0 else "56"
    proton_efficiency = strips_multitrack_efficiency[sector]
    proton_efficiency *= strips_sensor_efficiency[sector].GetBinContent(strips_sensor_efficiency[sector].FindBin(row["TrackX2"],row["TrackY2"]))
    proton_efficiency *= multiRP_efficiency[sector].GetBinContent(multiRP_efficiency[sector].FindBin(row["TrackX1"],row["TrackY1"]))
    efficiency.append(proton_efficiency)
    
    print("Sector "+sector+": Xi =",row["Xi"],"T =",row["T"],"Efficiency =", proton_efficiency)

Sector 56: Xi = 0.06730585545301437 T = -0.16012121737003326 Efficiency = 0.4609284998044084
Sector 56: Xi = 0.10286904126405716 T = -0.03971867263317108 Efficiency = 0.4635353958764146
Sector 45: Xi = 0.03689742088317871 T = -0.3153899610042572 Efficiency = 0.4401557612884728
Sector 56: Xi = 0.04731651395559311 T = -0.2775542438030243 Efficiency = 0.42737638048367266
Sector 45: Xi = 0.029943250119686127 T = -0.6871207356452942 Efficiency = 0.4137402806243656
Sector 45: Xi = 0.0673639103770256 T = -0.47823524475097656 Efficiency = 0.47139693092947915
Sector 56: Xi = 0.09101748466491699 T = -0.04628594592213631 Efficiency = 0.4741223853591719
Sector 45: Xi = 0.03961410000920296 T = -0.8172770142555237 Efficiency = 0.4562815265117985
Sector 56: Xi = 0.1573716253042221 T = -2.1758053302764893 Efficiency = 0.452176462651131
Sector 56: Xi = 0.052213169634342194 T = -0.3384019434452057 Efficiency = 0.4568379052128775
Sector 56: Xi = 0.045295294374227524 T = -1.1559875011444092 Efficiency = 0

Sector 56: Xi = 0.1342516988515854 T = -0.05550878122448921 Efficiency = 0.47631531354578693
Sector 45: Xi = 0.09491238743066788 T = -0.5244734287261963 Efficiency = 0.46323675404902787
Sector 45: Xi = 0.08448734879493713 T = -0.08706109970808029 Efficiency = 0.4704218080145913
Sector 45: Xi = 0.04044327139854431 T = -0.2427908331155777 Efficiency = 0.45022732069068766
Sector 56: Xi = 0.14125993847846985 T = -0.6829843521118164 Efficiency = 0.4859859290151861
Sector 56: Xi = 0.05784069374203682 T = -0.10368449240922928 Efficiency = 0.4536608387640148
Sector 45: Xi = 0.07924777269363403 T = -0.04334797337651253 Efficiency = 0.46861864960708943
Sector 56: Xi = 0.11020191758871078 T = -0.21690936386585236 Efficiency = 0.4692578193030583
Sector 45: Xi = 0.028940793126821518 T = -0.07942982017993927 Efficiency = 0.22392776938019524
Sector 45: Xi = 0.04401745647192001 T = -0.43041524291038513 Efficiency = 0.4315935259941891
Sector 45: Xi = 0.05596942454576492 T = -0.4627600312232971 Efficien

Sector 56: Xi = 0.13223370909690857 T = -1.2366377115249634 Efficiency = 0.45233748032604243
Sector 56: Xi = 0.05560589209198952 T = -0.08990022540092468 Efficiency = 0.4296651401722007
Sector 56: Xi = 0.061401739716529846 T = -1.2157342433929443 Efficiency = 0.46249416007386696
Sector 56: Xi = 0.03134329989552498 T = -0.4426354765892029 Efficiency = 0.39770475075912354
Sector 56: Xi = 0.038205597549676895 T = -0.11549969017505646 Efficiency = 0.43015433397562397
Sector 56: Xi = 0.08667450398206711 T = -0.22851130366325378 Efficiency = 0.45767384006027195
Sector 45: Xi = 0.035946909338235855 T = -0.7249810099601746 Efficiency = 0.47471448497270097
Sector 56: Xi = 0.10973659157752991 T = -0.6445392966270447 Efficiency = 0.4753498882377137
Sector 56: Xi = 0.1074504554271698 T = -0.21060477197170258 Efficiency = 0.4595817387884213
Sector 45: Xi = 0.0863465890288353 T = -0.16279716789722443 Efficiency = 0.4739478469436534
Sector 56: Xi = 0.05310515686869621 T = -0.16122691333293915 Efficie

Sector 45: Xi = 0.11201242357492447 T = -0.06488243490457535 Efficiency = 0.4762211568500639
Sector 45: Xi = 0.11337223649024963 T = -0.023876026272773743 Efficiency = 0.49291163117732634
Sector 45: Xi = 0.08669447153806686 T = -0.12639360129833221 Efficiency = 0.4705957310998875
Sector 45: Xi = 0.048355720937252045 T = -0.21451501548290253 Efficiency = 0.46007365003813516
Sector 45: Xi = 0.07149854302406311 T = -0.11624247580766678 Efficiency = 0.468610934099652
Sector 56: Xi = 0.05742618441581726 T = -173.2928924560547 Efficiency = 0.01615255277246163
Sector 45: Xi = 0.0865253359079361 T = -0.2238960713148117 Efficiency = 0.48272975495192116
Sector 45: Xi = 0.03273472934961319 T = -0.29700055718421936 Efficiency = 0.41247163024268907
Sector 45: Xi = 0.052152279764413834 T = -0.10611765831708908 Efficiency = 0.4470882885700785
Sector 45: Xi = 0.08058615028858185 T = -0.03546184301376343 Efficiency = 0.4630862351060326
Sector 45: Xi = 0.08014407753944397 T = -0.2426183670759201 Efficie

Sector 45: Xi = 0.09115087985992432 T = -0.04527377709746361 Efficiency = 0.47964357906630983
Sector 45: Xi = 0.033193159848451614 T = -0.5465712547302246 Efficiency = 0.37695071878754777
Sector 45: Xi = 0.02999749779701233 T = -0.5939980745315552 Efficiency = 0.3722402368837208
Sector 45: Xi = 0.03330496326088905 T = -0.0963684469461441 Efficiency = 0.3936599478315007
Sector 45: Xi = 0.081571064889431 T = -1.2727280855178833 Efficiency = 0.4292855176006214
Sector 45: Xi = 0.031392160803079605 T = -0.0712677463889122 Efficiency = 0.24548921560288015
Sector 45: Xi = 0.09709350764751434 T = -0.29346582293510437 Efficiency = 0.4629385864830784
Sector 45: Xi = 0.06681463867425919 T = -0.27429479360580444 Efficiency = 0.45899055072122746
Sector 56: Xi = 0.04392176866531372 T = -0.4244956076145172 Efficiency = 0.43094329893397776
Sector 45: Xi = 0.03814101591706276 T = -0.058479465544223785 Efficiency = 0.24149356005619937
Sector 56: Xi = 0.12999820709228516 T = -0.47333842515945435 Efficien

...

Sector 45: Xi = 0.0557355061173439 T = -0.042921166867017746 Efficiency = 0.4637896478554838
Sector 56: Xi = 0.06487354636192322 T = -0.3691098988056183 Efficiency = 0.45930457865393703
Sector 56: Xi = 0.07506882399320602 T = -0.4542946219444275 Efficiency = 0.468053245667926
Sector 45: Xi = 0.11152508109807968 T = -0.086955226957798 Efficiency = 0.4787229515866954
Sector 56: Xi = 0.0460781492292881 T = -0.17016683518886566 Efficiency = 0.43027137178505764
Sector 45: Xi = 0.10392632335424423 T = -0.40694573521614075 Efficiency = 0.4768559735915289
Sector 56: Xi = 0.09009765833616257 T = -0.02831035479903221 Efficiency = 0.46195680833922653
Sector 45: Xi = 0.04944046959280968 T = -0.016057420521974564 Efficiency = 0.45024219159543794
Sector 45: Xi = 0.11222716420888901 T = -0.13469290733337402 Efficiency = 0.48874498702123065
Sector 56: Xi = 0.12793192267417908 T = -0.10928086191415787 Efficiency = 0.47682118148951436
Sector 45: Xi = 0.06894318759441376 T = -0.283820778131485 Efficiency

Sector 45: Xi = 0.09193149954080582 T = -0.010645024478435516 Efficiency = 0.4708491923551898
Sector 56: Xi = 0.11906185001134872 T = -0.1503521353006363 Efficiency = 0.47077477094760256
Sector 45: Xi = 0.06130169332027435 T = -1.3151302337646484 Efficiency = 0.45092228766137943
Sector 45: Xi = 0.10078981518745422 T = -0.28697267174720764 Efficiency = 0.4903985671191903
Sector 45: Xi = 0.07613707333803177 T = -0.049140289425849915 Efficiency = 0.466249958117785
Sector 56: Xi = 0.1218714565038681 T = -0.5227286219596863 Efficiency = 0.479905238092293
Sector 45: Xi = 0.07406360656023026 T = -0.06367821991443634 Efficiency = 0.46588778174316986
Sector 45: Xi = 0.05351901054382324 T = -0.9603804349899292 Efficiency = 0.4201479238600013
Sector 56: Xi = 0.10359024256467819 T = -0.03808499127626419 Efficiency = 0.4663852481861122
Sector 45: Xi = 0.10422593355178833 T = -0.023923590779304504 Efficiency = 0.4829906592353352
Sector 56: Xi = 0.08213961124420166 T = -0.12715990841388702 Efficiency

Sector 45: Xi = 0.0943324938416481 T = -0.030968330800533295 Efficiency = 0.48292085321004785
Sector 56: Xi = 0.04492667689919472 T = -1.1383743286132812 Efficiency = 0.41062855841824736
Sector 56: Xi = 0.1247098445892334 T = -0.9878109097480774 Efficiency = 0.495549843732222
Sector 45: Xi = 0.11525534093379974 T = -0.9001813530921936 Efficiency = 0.4800259870801553
Sector 56: Xi = 0.040923479944467545 T = -1.2935867309570312 Efficiency = 0.4309612936210373
Sector 56: Xi = 0.08273214846849442 T = -0.6881532073020935 Efficiency = 0.496223294347391
Sector 45: Xi = 0.039427973330020905 T = -0.0609307736158371 Efficiency = 0.4398383834674633
Sector 56: Xi = 0.09062154591083527 T = -0.1323087215423584 Efficiency = 0.5057049344309683
Sector 45: Xi = 0.09134582430124283 T = -0.18104995787143707 Efficiency = 0.4673390132376045
Sector 56: Xi = 0.13707004487514496 T = -0.3550986349582672 Efficiency = 0.4737806938561741
Sector 56: Xi = 0.05529135838150978 T = -0.27045392990112305 Efficiency = 0.4

Sector 45: Xi = 0.12076739966869354 T = -0.020812351256608963 Efficiency = 0.48126185345817124
Sector 56: Xi = 0.09642520546913147 T = -1.0709623098373413 Efficiency = 0.4055025779656861
Sector 56: Xi = 0.13559438288211823 T = -1.45351243019104 Efficiency = 0.47903061118032725
Sector 56: Xi = 0.1050296425819397 T = -0.17155058681964874 Efficiency = 0.46567418008296957
Sector 45: Xi = 0.1656455248594284 T = -2.5688583850860596 Efficiency = 0.2400568620586735
Sector 56: Xi = 0.0421418659389019 T = -0.1257597953081131 Efficiency = 0.3331501838565596
Sector 56: Xi = 0.060670144855976105 T = -0.14767959713935852 Efficiency = 0.43439171521075054
Sector 45: Xi = 0.07169771939516068 T = -0.0209884624928236 Efficiency = 0.46309420811266305
Sector 45: Xi = 0.0408087894320488 T = -0.4499228298664093 Efficiency = 0.4484557943005375
Sector 56: Xi = 0.03567397966980934 T = -0.17696012556552887 Efficiency = 0.32626524596262924
Sector 45: Xi = 0.12007952481508255 T = -0.13866078853607178 Efficiency = 

Sector 56: Xi = 0.1014431044459343 T = -0.14460580050945282 Efficiency = 0.46719397303911886
Sector 56: Xi = 0.09938967972993851 T = -0.11860576272010803 Efficiency = 0.4597252818503894
Sector 45: Xi = 0.04920528084039688 T = -0.18516981601715088 Efficiency = 0.45998980727942845
Sector 45: Xi = 0.0754619762301445 T = -1.1292493343353271 Efficiency = 0.47563055470151505
Sector 56: Xi = 0.12495779991149902 T = -0.4279516339302063 Efficiency = 0.48376276364083487
Sector 56: Xi = 0.10718920826911926 T = -0.5349949598312378 Efficiency = 0.4532151363888256
Sector 45: Xi = 0.07818412780761719 T = -0.32643234729766846 Efficiency = 0.4545637872672241
Sector 56: Xi = 0.05625530332326889 T = -0.5706654787063599 Efficiency = 0.45237135099649484
Sector 56: Xi = 0.10514526069164276 T = -0.08843211829662323 Efficiency = 0.46199581647214644
Sector 45: Xi = 0.03199898824095726 T = -0.45990481972694397 Efficiency = 0.4283910001496357
Sector 45: Xi = 0.08010538667440414 T = -0.3735570013523102 Efficiency

In [18]:
# Add efficiency column to the new multiRP protons dataframe
df_multiRP_withEfficiency = df_multiRP.copy()
df_multiRP_withEfficiency["Efficiency"] = efficiency 

In [19]:
# Show a few lines of the new dataframe
df_multiRP_withEfficiency[:20]

,Run,LumiSection,EventNum,CrossingAngle,MultiRP,Arm,RPId1,RPId2,TrackX1,TrackY1,...,ThX,ThY,Muon0Pt,Muon1Pt,InvMass,ExtraPfCands,Acopl,XiMuMuPlus,XiMuMuMinus,Efficiency
3,297227,55,38079658,120.0,1,1,103,123,4.577393,1.367676,...,-0.000061,-0.000014,63.340645,62.493637,123.616515,27,0.132612,0.001953,0.048346,0.460928
6,297227,56,38791639,120.0,1,1,103,123,8.585938,2.644531,...,-0.000015,0.000024,65.270454,65.022888,152.786010,29,0.058782,0.002211,0.062855,0.463535
11,297227,56,38604580,120.0,1,0,3,23,4.131104,0.797180,...,0.000015,-0.000087,74.027679,70.969543,90.916580,28,0.588286,0.003354,0.038210,0.440156
12,297227,56,38604580,120.0,1,1,103,123,3.514526,1.507568,...,-0.000034,-0.000076,74.027679,70.969543,90.916580,28,0.588286,0.003354,0.038210,0.427376
22,297227,59,40601157,120.0,1,0,3,23,3.521118,0.645874,...,0.000058,-0.000116,53.758015,50.114189,105.095469,20,0.052973,0.025162,0.002618,0.413740
33,297227,70,49904103,120.0,1,0,3,23,9.614258,1.467651,...,0.000109,-0.000016,60.526527,53.105831,93.213809,80,0.665581,0.003924,0.027699,0.471397
36,297227,70,49482509,120.0,1,1,103,123,8.175293,1.603638,...,0.000012,0.000029,88.013977,52.118660,91.549497,21,0.527584,0.007379,0.015748,0.474122
39,297227,71,50687027,120.0,1,0,3,23,5.214844,-0.371826,...,0.000107,0.000093,126.357605,113.611771,270.992400,27,0.016921,0.010428,0.041786,0.456282
43,297227,73,51787113,120.0,1,1,103,123,4.165283,2.669922,...,-0.000208,0.000130,169.709396,123.770248,96.469760,13,0.930855,0.059182,0.009443,0.452176
48,297227,73,52238047,120.0,1,1,103,123,5.372803,0.135132,...,0.000086,0.000031,86.516212,56.114479,138.904508,10,0.060892,0.014483,0.008333,0.456838
